In [46]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [47]:
from typing import Union, Optional
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import re

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.datasets import fetch_dataset
import numpy as np
import pandas as pd
from astral.sun import sun
from astral import LocationInfo
from skforecast.datasets import fetch_dataset
from skforecast.model_selection import select_features


In [48]:
# Downloading data
# ==============================================================================
data = fetch_dataset('bike_sharing', raw=True)

# Preprocessing data (setting index and frequency)
# ==============================================================================
data = data[['date_time', 'users', 'holiday', 'weather', 'temp', 'atemp', 'hum', 'windspeed']]
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('date_time')
data = data.asfreq('H')
data = data.sort_index()
data.head()

# Calendar features
# ==============================================================================
calendar_features = pd.DataFrame(index=data.index)
calendar_features['month'] = calendar_features.index.month
calendar_features['week_of_year'] = calendar_features.index.isocalendar().week
calendar_features['week_day'] = calendar_features.index.day_of_week + 1
calendar_features['hour_day'] = calendar_features.index.hour + 1

# Sunlight features
# ==============================================================================
location = LocationInfo(
    name='Washington DC',
    region='USA',
    timezone='US/Eastern',
    latitude=40.516666666666666,
    longitude=-77.03333333333333
)
sunrise_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour
    for date in data.index
]
sunset_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour
    for date in data.index
]
sun_light_features = pd.DataFrame({
                         'sunrise_hour': sunrise_hour,
                         'sunset_hour': sunset_hour}, 
                         index = data.index
                     )
sun_light_features['daylight_hours'] = (
    sun_light_features['sunset_hour'] - sun_light_features['sunrise_hour']
)
sun_light_features['is_daylight'] = np.where(
                                        (data.index.hour >= sun_light_features['sunrise_hour']) & \
                                        (data.index.hour < sun_light_features['sunset_hour']),
                                        1,
                                        0
                                    )

# Holiday features
# ==============================================================================
holiday_features = data[['holiday']].astype(int)
holiday_features['holiday_previous_day'] = holiday_features['holiday'].shift(24)
holiday_features['holiday_next_day'] = holiday_features['holiday'].shift(-24)

# Temperature features
# ==============================================================================
temp_features = data[['temp']].copy()
temp_features['temp_roll_mean_1_day'] = temp_features['temp'].rolling(24, closed='left').mean()
temp_features['temp_roll_mean_7_day'] = temp_features['temp'].rolling(24*7, closed='left').mean()
temp_features['temp_roll_max_1_day'] = temp_features['temp'].rolling(24, closed='left').max()
temp_features['temp_roll_min_1_day'] = temp_features['temp'].rolling(24, closed='left').min()
temp_features['temp_roll_max_7_day'] = temp_features['temp'].rolling(24*7, closed='left').max()
temp_features['temp_roll_min_7_day'] = temp_features['temp'].rolling(24*7, closed='left').min()


# Merge all exogenous variables
# ==============================================================================
df_exogenous_features = pd.concat([
                            calendar_features,
                            sun_light_features,
                            temp_features,
                            holiday_features
                        ], axis=1)

df_exogenous_features.head(4)

# Cliclical encoding of calendar and sunlight features
# ==============================================================================
def cyclical_encoding(data: pd.Series, cycle_length: int) -> pd.DataFrame:
    """
    Encode a cyclical feature with two new features sine and cosine.
    The minimum value of the feature is assumed to be 0. The maximum value
    of the feature is passed as an argument.
      
    Parameters
    ----------
    data : pd.Series
        Series with the feature to encode.
    cycle_length : int
        The length of the cycle. For example, 12 for months, 24 for hours, etc.
        This value is used to calculate the angle of the sin and cos.

    Returns
    -------
    result : pd.DataFrame
        Dataframe with the two new features sin and cos.

    """

    sin = np.sin(2 * np.pi * data/cycle_length)
    cos = np.cos(2 * np.pi * data/cycle_length)
    result =  pd.DataFrame({
                  f"{data.name}_sin": sin,
                  f"{data.name}_cos": cos
              })

    return result


month_encoded = cyclical_encoding(df_exogenous_features['month'], cycle_length=12)
week_of_year_encoded = cyclical_encoding(df_exogenous_features['week_of_year'], cycle_length=52)
week_day_encoded = cyclical_encoding(df_exogenous_features['week_day'], cycle_length=7)
hour_day_encoded = cyclical_encoding(df_exogenous_features['hour_day'], cycle_length=24)
sunrise_hour_encoded = cyclical_encoding(df_exogenous_features['sunrise_hour'], cycle_length=24)
sunset_hour_encoded = cyclical_encoding(df_exogenous_features['sunset_hour'], cycle_length=24)

cyclical_features = pd.concat([
                        month_encoded,
                        week_of_year_encoded,
                        week_day_encoded,
                        hour_day_encoded,
                        sunrise_hour_encoded,
                        sunset_hour_encoded
                    ], axis=1)

df_exogenous_features = pd.concat([df_exogenous_features, cyclical_features], axis=1)
df_exogenous_features.head(3)

# Interaction between exogenous variables
# ==============================================================================
transformer_poly = PolynomialFeatures(
                       degree           = 2,
                       interaction_only = True,
                       include_bias     = False
                   ).set_output(transform="pandas")

poly_cols = [
    'month_sin', 
    'month_cos',
    'week_of_year_sin',
    'week_of_year_cos',
    'week_day_sin',
    'week_day_cos',
    'hour_day_sin',
    'hour_day_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'daylight_hours',
    'is_daylight',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day',
    'temp',
    'holiday'
]

poly_features = transformer_poly.fit_transform(df_exogenous_features[poly_cols].dropna())
poly_features = poly_features.drop(columns=poly_cols)
poly_features.columns = [f"poly_{col}" for col in poly_features.columns]
poly_features.columns = poly_features.columns.str.replace(" ", "__")
df_exogenous_features = pd.concat([df_exogenous_features, poly_features], axis=1)
df_exogenous_features.head(4)

# Select exogenous variables to be included in the model
# ==============================================================================
exog_features = []
# Columns that ends with _sin or _cos are selected
exog_features.extend(df_exogenous_features.filter(regex='_sin$|_cos$').columns.tolist())
# columns that start with temp_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^temp_.*').columns.tolist())
# Columns that start with holiday_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^holiday_.*').columns.tolist())
exog_features.extend(['temp', 'holiday'])

df_exogenous_features = df_exogenous_features.filter(exog_features, axis=1)

bike_sharing
------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, information
about weather conditions and holidays is available.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17544, 12)


In [49]:
df_exogenous_features = df_exogenous_features.dropna()
data = data.loc[df_exogenous_features.index, :]
df_exogenous_features['dummy_variable'] = 1

In [80]:
forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 100,
            )

selector = RFECV(
    estimator              = forecaster.regressor,
    min_features_to_select = 1,
    cv                     = 3
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    select_only_exog     = False,
    subsample            = 0.1,
    force_inclusion      = "^dummy_variable",
    verbose              = True,
)        

Recursive feature elimination
-----------------------------
Total number of features available: 188
Total number of records available: 17252
Total number of records used for feature selection: 1725
Number of features selected: 133
Selected lags: [1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 23, 24, 25, 31, 32, 35, 36, 38, 43, 46, 47, 49, 50, 51, 55, 57, 59, 60, 62, 70, 71, 72, 73, 74, 75, 82, 84, 85, 91, 96, 97]
Selected exog : 
 ['month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'poly_month_sin__month_cos', 'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos', 'poly_month_sin__week_day_sin', 'poly_month_sin__week_day_cos', 'poly_month_sin__hour_day_sin', 'poly_month_sin__hour_day_cos', 'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos', 'poly_month_sin__sunset_hour_sin', 'poly_month_s

In [82]:
def create_predictors(y):
    lags = y[-1:-6:-1]                
    return lags


forecaster = ForecasterAutoregCustom(
                regressor = Ridge(random_state=123),
                fun_predictors = create_predictors,
                window_size = 5,
            )

selector = RFECV(
    estimator              = forecaster.regressor,
    min_features_to_select = 1,
    cv                     = 3
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    select_only_exog     = False,
    subsample            = 0.1,
    force_inclusion      = "^dummy_variable",
    verbose              = True,
)  

Recursive feature elimination
-----------------------------
Total number of features available: 88
Total number of records available: 17347
Total number of records used for feature selection: 1734
Number of features selected: 78
Selected lags: ['custom_predictor_0', 'custom_predictor_1', 'custom_predictor_2', 'custom_predictor_3', 'custom_predictor_4']
Selected exog : 
 ['month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'hour_day_sin', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'poly_month_sin__month_cos', 'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos', 'poly_month_sin__week_day_sin', 'poly_month_sin__hour_day_sin', 'poly_month_sin__hour_day_cos', 'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos', 'poly_month_sin__sunset_hour_sin', 'poly_month_sin__sunset_hour_cos', 'poly_month_cos__week_of_year_sin', 'poly_month_cos__week_of_year_cos', 'poly_month_cos__week_day_sin', 'poly_m

In [86]:
from sklearn.feature_selection import SelectFromModel

forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 100,
            )

selector = SelectFromModel(
    estimator    = forecaster.regressor,
    threshold    = 0.25,
    max_features = 25
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    select_only_exog     = False,
    subsample            = 0.1,
    force_inclusion      = "^dummy_variable",
    verbose              = True,
)  

Recursive feature elimination
-----------------------------
Total number of features available: 188
Total number of records available: 17252
Total number of records used for feature selection: 1725
Number of features selected: 100
Selected lags: [1, 2, 9, 10, 17, 23, 24, 25, 43, 72, 74, 96, 97]
Selected exog : 
 ['month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'poly_month_sin__month_cos', 'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos', 'poly_month_sin__week_day_sin', 'poly_month_sin__week_day_cos', 'poly_month_sin__hour_day_sin', 'poly_month_sin__hour_day_cos', 'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos', 'poly_month_sin__sunset_hour_sin', 'poly_month_sin__sunset_hour_cos', 'poly_month_cos__week_of_year_sin', 'poly_month_cos__week_of_year_cos', 'poly_month_cos__week_day_sin', 'p

In [78]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import ShuffleSplit

forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 10,
            )

selector = SequentialFeatureSelector(
    estimator    = forecaster.regressor,
    n_features_to_select = 25,
    direction    = 'forward',
    cv           = ShuffleSplit(n_splits=1, test_size=0.3, random_state=951),
    scoring      = 'neg_mean_absolute_error',
    n_jobs       = -1
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    select_only_exog     = False,
    subsample            = 0.1,
    force_inclusion      = "^dummy_variable",
    verbose              = True,
)  

Index(['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8',
       'lag_9', 'lag_10', 'month_sin', 'month_cos', 'week_of_year_sin',
       'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin',
       'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos',
       'sunset_hour_sin', 'sunset_hour_cos', 'poly_month_sin__month_cos',
       'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos',
       'poly_month_sin__week_day_sin', 'poly_month_sin__week_day_cos',
       'poly_month_sin__hour_day_sin', 'poly_month_sin__hour_day_cos',
       'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos',
       'poly_month_sin__sunset_hour_sin', 'poly_month_sin__sunset_hour_cos',
       'poly_month_cos__week_of_year_sin', 'poly_month_cos__week_of_year_cos',
       'poly_month_cos__week_day_sin', 'poly_month_cos__week_day_cos',
       'poly_month_cos__hour_day_sin', 'poly_month_cos__hour_day_cos',
       'poly_month_cos__sunrise_hour_si

KeyboardInterrupt: 

In [148]:
# Simulate time series with hourly frequency with reproducible results
# ==============================================================================
from sklearn.feature_selection import RFE
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
import pytest

exog, y = make_regression(n_samples=500, n_features=5, n_informative=2, random_state=123)
exog = pd.DataFrame(
    exog,
    index = pd.date_range(start='2020-01-01', periods=len(exog), freq='H'),
    columns=[f"exog_{i}" for i in range(exog.shape[1])]
)
y = pd.Series(y, index=exog.index, name="y")

In [154]:
def test_select_features_when_selector_is_RFE_and_select_only_exog_is_True():
    forecaster = ForecasterAutoreg(
                    regressor = LinearRegression(),
                    lags      = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        verbose              = False,
    )

    assert selected_lags == [1, 2, 3, 4, 5]
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4']


def test_select_features_when_selector_is_RFE_and_select_only_exog_is_True_ForecasterAutoregCustom():
    forecaster = ForecasterAutoregCustom(
                    regressor = LinearRegression(),
                    fun_predictors = lambda y: y[-1:-6:-1],
                    window_size = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        verbose              = False,
    )

    assert selected_lags == ['custom_predictor_0', 'custom_predictor_1', 'custom_predictor_2',
                            'custom_predictor_3', 'custom_predictor_4']
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4']


def test_select_features_when_selector_is_RFE_and_select_only_exog_is_False():
    forecaster = ForecasterAutoreg(
                    regressor = LinearRegression(),
                    lags      = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=5)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = False,
        verbose              = False,
    )

    assert selected_lags == [5]
    assert selected_exog == ['exog_0', 'exog_1', 'exog_2', 'exog_3']


def test_select_features_when_selector_is_RFE_and_select_only_exog_is_False_ForecasterAutoregCustom():
    forecaster = ForecasterAutoregCustom(
                    regressor = LinearRegression(),
                    fun_predictors = lambda y: y[-1:-6:-1],
                    window_size = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=5)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = False,
        verbose              = False,
    )

    assert selected_lags == ['custom_predictor_4']
    assert selected_exog == ['exog_0', 'exog_1', 'exog_2', 'exog_3']


def test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_regex():
    forecaster = ForecasterAutoreg(
                    regressor = LinearRegression(),
                    lags      = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        force_inclusion      = "^exog_3",
        verbose              = False,
    )

    assert selected_lags == [1, 2, 3, 4, 5]
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4', 'exog_3']


def test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_regex_ForecasterAutoregCustom():
    forecaster = ForecasterAutoregCustom(
                    regressor = LinearRegression(),
                    fun_predictors = lambda y: y[-1:-6:-1],
                    window_size = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        force_inclusion      = "^exog_3",
        verbose              = False,
    )

    assert selected_lags == ['custom_predictor_0', 'custom_predictor_1', 'custom_predictor_2',
                            'custom_predictor_3', 'custom_predictor_4']
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4', 'exog_3']


def test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_list():
    forecaster = ForecasterAutoreg(
                    regressor = LinearRegression(),
                    lags      = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        force_inclusion      = ['exog_3'],
        verbose              = False,
    )

    assert selected_lags == [1, 2, 3, 4, 5]
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4', 'exog_3']


def test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_list_ForecasterAutoregCustom():
    forecaster = ForecasterAutoregCustom(
                    regressor = LinearRegression(),
                    fun_predictors = lambda y: y[-1:-6:-1],
                    window_size = 5,
                )

    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)

    selected_lags, selected_exog = select_features(
        selector             = selector,
        forecaster           = forecaster,
        y                    = y,
        exog                 = exog,
        select_only_exog     = True,
        force_inclusion      = ['exog_3'],
        verbose              = False,
    )

    assert selected_lags == ['custom_predictor_0', 'custom_predictor_1', 'custom_predictor_2',
                            'custom_predictor_3', 'custom_predictor_4']
    assert selected_exog == ['exog_1', 'exog_2', 'exog_4', 'exog_3']

def test_select_features_raise_error_when_forecaster_is_not_supported():
    """
    Test that select_features raises an error when forecaster is not supported.
    """
    forecaster = ForecasterAutoregDirect(
                    regressor = LinearRegression(),
                    lags      = 5,
                    steps     = 3
                )
    selector = RFE(estimator=forecaster.regressor, n_features_to_select=3)
    err_msg = re.escape(
            "`forecaster` must be one of the following classes: ['ForecasterAutoreg', "
            "'ForecasterAutoregCustom', 'ForecasterAutoregMultiSeries', "
            "'ForecasterAutoregMultiSeriesCustom']."
        )
    with pytest.raises(Exception, match = err_msg):
        selected_lags, selected_exog = select_features(
            selector             = selector,
            forecaster           = forecaster,
            y                    = y,
            exog                 = exog,
            select_only_exog     = True,
            verbose              = False,
        )

test_select_features_when_selector_is_RFE_and_select_only_exog_is_True()
test_select_features_when_selector_is_RFE_and_select_only_exog_is_True_ForecasterAutoregCustom()
test_select_features_when_selector_is_RFE_and_select_only_exog_is_False()
test_select_features_when_selector_is_RFE_and_select_only_exog_is_False_ForecasterAutoregCustom()
test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_regex()
test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_regex_ForecasterAutoregCustom()
test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_list()
test_select_features_when_selector_is_RFE_select_only_exog_is_True_and_force_inclusion_is_list_ForecasterAutoregCustom()
test_select_features_raise_error_when_forecaster_is_not_supported()